In [1]:
import tensorflow as tf
from tensorflow.keras.models import clone_model
import matplotlib.pyplot as plt
import numpy as np

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model1 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# Trial 1 batchsize 32 vs 256
epochs = 10

model1.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.fit(x_train, y_train, epochs=epochs, batch_size=32)

model2 = clone_model(model1)
model2.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.fit(x_train, y_train, epochs=epochs, batch_size=256)


Epoch 1/10
60000/60000 [==============================] - 5s 91us/sample - loss: 0.6293 - acc: 0.8399
Epoch 2/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.3396 - acc: 0.9045
Epoch 3/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.2868 - acc: 0.9188
Epoch 4/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.2536 - acc: 0.9287
Epoch 5/10
60000/60000 [==============================] - 5s 86us/sample - loss: 0.2298 - acc: 0.9363
Epoch 6/10
60000/60000 [==============================] - 5s 88us/sample - loss: 0.2091 - acc: 0.9409
Epoch 7/10
60000/60000 [==============================] - 5s 88us/sample - loss: 0.1933 - acc: 0.9452
Epoch 8/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.1799 - acc: 0.9492
Epoch 9/10
60000/60000 [==============================] - 5s 83us/sample - loss: 0.1679 - acc: 0.9531
Epoch 10/10
60000/60000 [==============================] - 5s 86us/sample - loss: 

In [ ]:

weights1 = model1.get_weights()
weights2 = model2.get_weights()
pmodel = clone_model(model1)
pmodel.compile(optimizer=model1.optimizer, loss=model1.loss)


# structure: [alpha, train_loss, train_acc, test_loss, test_acc]
results = []

for i in range(-100, 201):
    alpha = i*0.01
    weights3 = [ (1-alpha)*t1 + alpha*t2 for t1, t2 in zip(weights1, weights2) ]
    pmodel.set_weights(weights3)
    train_loss, train_acc = pmodel.evaluate(x=x_train, y=y_train, verbose=0, batch_size=256)
    test_loss, test_acc = pmodel.evaluate(x=x_test, y=y_test, verbose=0, batch_size=256)
    results.append([alpha, train_loss, train_acc, test_loss, test_acc])
    
results = np.asarray(results)

In [ ]:
import matplotlib.pyplot as plt
# fig, ax1 = plt.subplots()

# plots the training loss
plt.plot(results[:,0], results[:,1])
# plots the testing loss
plt.plot(results[:,0], results[:,3], fmt=':')

plt.twinx()
# plots the training accu
plt.plot(results[:,0], results[:,2])
# plots the testing acc
plt.plot(results[:,0], results[:,4], fmt=':')
plt.xlabel('alpha')
plt.ylabel('alpha')